In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = Chain1, Chain2, Chain3, Chain4
Samples per chain = 1000
parameters        = a, bp, a_society.1, a_society.2, a_society.3, a_society.4, a_society.5, a_society.6, a_society.7, a_society.8, a_society.9, a_society.10, sigma_society

Empirical Posterior Estimates:
parameters
                Mean    SD   Naive SE  MCSE   ESS
            a  1.0912 0.7873   0.0124 0.0232 1000
  a_society.1 -0.2022 0.2464   0.0039 0.0061 1000
  a_society.2  0.0501 0.2292   0.0036 0.0058 1000
  a_society.3 -0.0410 0.1982   0.0031 0.0048 1000
  a_society.4  0.3404 0.1992   0.0031 0.0049 1000
  a_society.5  0.0505 0.1808   0.0029 0.0039 1000
  a_society.6 -0.3224 0.2067   0.0033 0.0057 1000
  a_society.7  0.1567 0.1784   0.0028 0.0039 1000
  a_society.8 -0.1687 0.1881   0.0030 0.0044 1000
  a_society.9  0.2836 0.1859   0.0029 0.0040 1000
 a_society.10 -0.0892 0.3145   0.0050 0.0090 1000
           bp  0.2612 0.0862   0.001

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*